In [7]:

import os # Para gerenciar variáveis de ambiente, se desejado


API_KEY_CONFIGURADA = False

def configurar_api_key():
    global API_KEY_CONFIGURADA
    if not API_KEY_CONFIGURADA:
        try:
            # Tenta pegar da variável de ambiente do Colab primeiro
            from google.colab import userdata
            key = userdata.get('GOOGLE_API_KEY') # Use um nome específico para a chave
            if key:
                genai.configure(api_key=key)
                API_KEY_CONFIGURADA = True
                print("✅ Chave de API configurada a partir dos secrets do Colab.\n")
                return True
        except ImportError:
            pass # Não está no Colab ou userdata não disponível

        # Se não configurada, pede ao usuário
        print("🔑 Por favor, insira sua Chave de API do Google AI Studio (Gemini):")
        key = input("API Key: ").strip()
        if key:
            genai.configure(api_key=key)
            API_KEY_CONFIGURADA = True
            print("✅ Chave de API configurada.\n")
            return True
        else:
            print("⚠️ Nenhuma chave de API fornecida. O chatbot não poderá usar os modelos Gemini.")
            return False
    return True


##########################################################################
# --- Definição da Estrutura do Agente (Conceitual) ---
# No seu exemplo, `Agent` e `call_agent` seriam de uma biblioteca.
# Aqui, vamos simular a ideia com funções e chamadas diretas ao SDK do Gemini.
##########################################################################

# Modelo LLM a ser usado pelos agentes.
# Modelos recomendados: "gemini-1.5-flash-latest" (rápido e eficiente) ou "gemini-1.5-pro-latest" (mais poderoso)
# O usuário mencionou "gemini-2.5-pro-preview-03-25", mas os nomes de modelo na API são como os acima.
# Adapte conforme os modelos disponíveis e suas preferências.
MODEL_NAME = "gemini-1.5-flash-latest"

generation_config = {
    "temperature": 0.7, # Controla a criatividade. Mais baixo = mais determinístico.
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192, # Ajuste conforme necessário
    "response_mime_type": "text/plain",
}

safety_settings = [ # Configurações de segurança para evitar conteúdo prejudicial
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

def chamar_agente_llm(nome_agente, instrucao_sistema, entrada_usuario, historico_chat=None):
    """
    Função para simular a chamada a um agente LLM.
    `instrucao_sistema` é o prompt que define o comportamento do agente.
    `entrada_usuario` é a pergunta ou informação específica para esta chamada.
    `historico_chat` (opcional) para manter o contexto da conversa.
    """
    if not API_KEY_CONFIGURADA:
        return "⚠️ Chave de API não configurada. Não posso contatar o modelo Gemini."

    print(f"🤖 Acionando {nome_agente} com modelo {MODEL_NAME}...")
    try:
        model = genai.GenerativeModel(
            model_name=MODEL_NAME,
            safety_settings=safety_settings,
            generation_config=generation_config,
            system_instruction=instrucao_sistema # Define o papel e contexto do agente
        )

        # Se houver histórico, formata para a API
        # A API `start_chat` é boa para conversas multi-turnos.
        # Para chamadas mais "stateless" de agentes específicos, `generate_content` com histórico no prompt pode ser usado.
        # Aqui, vamos usar `generate_content` e incluir a entrada do usuário no prompt.

        # Combina a entrada do usuário com o histórico, se houver, para dar contexto.
        # A instrução de sistema já dá o papel do agente.
        # O `entrada_usuario` é o que o agente deve processar AGORA.

        # Para um agente que só precisa da entrada atual para sua tarefa específica:
        conteudo_prompt = entrada_usuario

        if historico_chat: # Opcional, se quisermos que o agente lembre do histórico para esta tarefa
            # Formato simples de histórico:
            # historico_formatado = "\n\nHistórico da Conversa Anterior:\n"
            # for msg in historico_chat:
            #     historico_formatado += f"{msg['role']}: {msg['parts'][0]['text']}\n"
            # conteudo_prompt = historico_formatado + "\n\nTarefa Atual:\n" + entrada_usuario
            # Para o Gemini, o histórico é melhor gerenciado pela sessão de chat.
            # Mas como estamos fazendo chamadas "one-off" por agente, passamos o contexto na entrada.
            pass # Simplicidade por agora: `entrada_usuario` é o foco.

        response = model.generate_content(conteudo_prompt)
        return response.text.strip()
    except Exception as e:
        print(f"❌ Erro ao chamar o agente {nome_agente}: {e}")
        return f"⚠️ Desculpe, tive um problema ao processar sua solicitação com o agente {nome_agente}. Por favor, tente novamente."


# --- Estado da Conversa ---
estado_conversa = {}

def resetar_estado_conversa():
    global estado_conversa
    estado_conversa = {
        "desastre_atual": None,
        "localizacao_perguntada": False,
        "local_informado": None,
        "sugestoes_abrigo_dadas": False,
        "verificacao_seguranca_perguntada": False,
        "usuario_seguro_confirmado": None,
        "historico_interacao_agentes": [] # Para manter algum contexto se necessário
    }

resetar_estado_conversa() # Inicia com estado limpo

# --- Definição dos Agentes (com LLM) ---

def agente_roteador_intencao(texto_usuario):
    """
    Este agente (conceitual) tenta identificar o desastre, a urgência e a necessidade principal.
    Por simplicidade, ainda usaremos algumas keywords, mas um LLM poderia fazer isso de forma mais sofisticada.
    Retorna: (desastre_identificado, tipo_ajuda)
    tipo_ajuda pode ser: "imediata", "planejamento", "local_seguro", "status_seguranca", "geral", "desconhecida"
    """
    texto_lower = texto_usuario.lower()
    desastres_conhecidos = ["enchente", "inundação", "alagamento", "deslizamento", "soterramento", "incêndio florestal", "queimada"] # Adicionar sinônimos
    desastre_id = None
    for d in desastres_conhecidos:
        if d in texto_lower:
            if d in ["inundação", "alagamento"]:
                desastre_id = "enchente"
            elif d == "soterramento":
                desastre_id = "deslizamento"
            elif d == "queimada":
                desastre_id = "incêndio florestal"
            else:
                desastre_id = d
            break

    if not desastre_id and ("desastre" in texto_lower or "emergência" in texto_lower): # Se não identifica tipo, mas fala de desastre
        desastre_id = "desastre genérico" # Poderíamos pedir para especificar

    tipo_ajuda = "geral" # Default

    palavras_chave_imediato = ["ajuda", "socorro", "agora", "o que fazer", "estou em", "acontecendo", "urgente", "perigo", "durante"]
    palavras_chave_planejamento = ["preparar", "prevenção", "evitar", "depois", "informação", "riscos", "kit", "como me proteger", "plano", "antes"]
    palavras_chave_local_seguro = ["local seguro", "abrigo", "onde ir", "segurança", "protegido"]
    palavras_chave_status_seguranca = ["estou bem", "estou seguro", "não estou seguro", "consegui"]


    if any(p in texto_lower for p in palavras_chave_imediato):
        tipo_ajuda = "imediata"
    elif any(p in texto_lower for p in palavras_chave_planejamento):
        tipo_ajuda = "planejamento"
    elif any(p in texto_lower for p in palavras_chave_local_seguro) and estado_conversa.get("localizacao_perguntada"):
        # Se pergunta sobre local seguro APÓS a localização ser perguntada, provavelmente é uma resposta de localização
        # ou uma pergunta subsequente. O fluxo principal tratará isso.
        # Se pergunta sobre local seguro ANTES, o agente de abrigos pode ser chamado.
        tipo_ajuda = "local_seguro"
    elif any(p in texto_lower for p in palavras_chave_status_seguranca) and estado_conversa.get("verificacao_seguranca_perguntada"):
        tipo_ajuda = "status_seguranca"
    elif not desastre_id:
        tipo_ajuda = "desconhecida"

    return desastre_id, tipo_ajuda


def agente_resposta_rapida_llm(tipo_desastre):
    nome_agente = "Agente de Resposta Rápida"
    instrucao = f"""
        Você é um especialista em resposta a emergências de desastres naturais.
        Sua missão é fornecer ações IMEDIATAS e CRUCIAIS que uma pessoa deve tomar para se proteger durante um '{tipo_desastre}'.
        Seja claro, conciso, use marcadores (como '➡️') para cada ação e priorize a segurança.
        Ao final, SEMPRE inclua a frase: 'Lembre-se, em caso de emergência, ligue para a Defesa Civil (199) ou Corpo de Bombeiros (193).'
        Não adicione saudações ou despedidas, apenas as ações e o aviso dos números de emergência.
        Se o tipo de desastre for 'desastre genérico' ou não especificado, dê conselhos gerais para qualquer emergência (manter a calma, procurar informações oficiais, ir para local seguro, ter kit de emergência à mão).
    """
    entrada = f"Qual a ação imediata para {tipo_desastre}?"
    return chamar_agente_llm(nome_agente, instrucao, entrada)

def agente_planejamento_suporte_llm(tipo_desastre):
    nome_agente = "Agente de Planejamento e Suporte"
    instrucao = f"""
        Você é um especialista em preparação para desastres naturais e suporte pós-desastre.
        Sua tarefa é fornecer informações detalhadas sobre:
        1. Como se PREPARAR para um '{tipo_desastre}'.
        2. O que incluir em um KIT DE EMERGÊNCIA essencial.
        3. Considerações importantes para o período APÓS o desastre.
        Use linguagem clara e organize as informações com títulos ou marcadores (como '➡️').
        Incentive o planejamento familiar e a busca por informações da Defesa Civil local.
        Não adicione saudações ou despedidas.
        Se o tipo de desastre for 'desastre genérico', forneça dicas gerais de preparação e o que fazer após qualquer emergência.
    """
    entrada = f"Como se preparar, o que ter no kit de emergência e o que fazer após um {tipo_desastre}?"
    return chamar_agente_llm(nome_agente, instrucao, entrada)

def agente_localizacao_perguntar_txt(): # Este não precisa de LLM para perguntar
    global estado_conversa
    estado_conversa["localizacao_perguntada"] = True
    return [
        "--- 📍 Agente de Localização ---",
        "Para que eu possa fornecer sugestões mais personalizadas (de forma genérica, pois sou um IA), por favor, informe sua cidade/bairro ou um ponto de referência.",
        "Lembre-se: esta é uma simulação. Não compartilhe dados pessoais ou localização exata e precisa se não se sentir confortável."
    ]

def agente_abrigos_sugerir_llm(tipo_desastre, local_usuario_simulado):
    global estado_conversa
    nome_agente = "Agente de Abrigos Seguros"
    instrucao_sistema = f"""
        Você é um assistente de segurança especializado em desastres naturais.
        Sua tarefa é fornecer SUGESTÕES GERAIS sobre tipos de locais que costumam ser mais seguros ou podem servir como abrigos durante um '{tipo_desastre}', considerando a localização aproximada do usuário em '{local_usuario_simulado if local_usuario_simulado else "local não informado"}'.

        IMPORTANTÍSSIMO:
        1.  ENFATIZE que o usuário DEVE, PRIORITARIAMENTE, procurar informações sobre ABRIGOS OFICIAIS designados pelas autoridades locais (Defesa Civil, Prefeitura).
        2.  DEIXE CLARO que suas sugestões são GENÉRICAS e NÃO substituem as orientações oficiais e em tempo real.
        3.  NÃO INVENTE nomes de abrigos específicos (ruas, nomes de escolas) a menos que seja um conhecimento público amplamente divulgado para esse fim em grandes cidades (e mesmo assim, com cautela).
        4.  Se a localização for muito genérica ou não informada, forneça conselhos gerais sobre como identificar lugares seguros para aquele tipo de desastre.
        5.  Use marcadores (como '➡️') para cada sugestão.
        6.  Se o tipo de desastre for 'desastre genérico', aconselhe a procurar locais altos e estruturalmente seguros, longe de perigos óbvios, e a seguir as rotas de evacuação indicadas pelas autoridades.
        Finalize lembrando o usuário de verificar fontes oficiais. Não adicione saudações.
    """
    entrada_prompt = f"Sugestões de locais seguros para {tipo_desastre} na área de {local_usuario_simulado if local_usuario_simulado else 'local não informado'}."
    resposta = chamar_agente_llm(nome_agente, instrucao_sistema, entrada_prompt)
    if "⚠️" not in resposta: # Se não for uma mensagem de erro da chamada do LLM
        estado_conversa["sugestoes_abrigo_dadas"] = True
    return resposta


def agente_verificacao_seguranca_perguntar_txt(): # Este não precisa de LLM para perguntar
    global estado_conversa
    estado_conversa["verificacao_seguranca_perguntada"] = True
    return ["--- ✅ Agente de Verificação de Segurança ---", "Você conseguiu encontrar ou se dirigir a um local que considera seguro neste momento? (responda com sim/não ou descreva sua situação)"]

def agente_processar_resposta_seguranca_llm(resposta_usuario_seguranca):
    global estado_conversa
    nome_agente = "Agente de Suporte Emocional e Próximos Passos" # Nome mais descritivo
    instrucao_sistema = f"""
        O usuário respondeu à pergunta 'Você conseguiu encontrar um local seguro?'.
        A resposta do usuário foi: '{resposta_usuario_seguranca}'.
        Sua tarefa é:
        1.  Interpretar a resposta do usuário (se ele está seguro, parcialmente seguro, não seguro, ou incerto).
        2.  Fornecer uma mensagem de reconhecimento e encorajamento adequada à situação.
        3.  Se o usuário indicar que NÃO está seguro ou está em perigo, REFORCE a importância de buscar segurança IMEDIATAMENTE e, se possível e necessário, contatar os serviços de emergência (Defesa Civil 199, Bombeiros 193).
        4.  Se o usuário estiver seguro, parabenize-o e sugira que continue atento às informações oficiais.
        Seja empático e direto. Não faça perguntas adicionais a menos que seja para clarificar o status de segurança se a resposta for muito ambígua.
    """
    entrada_prompt = f"Analisar e responder à seguinte afirmação do usuário sobre sua segurança: {resposta_usuario_seguranca}"
    resposta_llm = chamar_agente_llm(nome_agente, instrucao_sistema, entrada_prompt)

    # Tenta uma lógica simples para atualizar o estado_conversa com base na resposta do LLM ou da entrada do usuário
    # Isso é uma simplificação; um LLM poderia classificar o status de segurança de forma mais robusta.
    resposta_lower = resposta_usuario_seguranca.lower()
    if any(s in resposta_lower for s in ["sim", "estou seguro", "consegui", "estou bem"]):
        estado_conversa["usuario_seguro_confirmado"] = True
    elif any(s in resposta_lower for s in ["não", "nao", "perigo", "ajuda ainda", "preso"]):
        estado_conversa["usuario_seguro_confirmado"] = False
    # Se ambíguo, `usuario_seguro_confirmado` permanece None ou como o LLM interpretou.

    return resposta_llm

# --- Lógica Principal do Chatbot (Adaptada para LLMs) ---
def chatbot_desastres_naturais_llm():
    if not configurar_api_key():
        return # Sai se a API key não for configurada

    print("\n---")
    print("🤖 Bem-vindo ao Chatbot Inteligente de Assistência em Desastres Naturais!")
    print("   Sou um assistente virtual e usarei IA para tentar te ajudar.")
    print("   Lembre-se: Minhas informações são um auxílio. SEMPRE siga as orientações")
    print("   das autoridades oficiais (Defesa Civil, Bombeiros).")
    print("---")
    print("Como posso te ajudar hoje? (Para sair, digite 'sair')")

    resetar_estado_conversa()

    while True:
        usuario_input = input("\n🙎 Você: ").strip()
        if usuario_input.lower() == "sair":
            print("\n🤖 Chatbot: Obrigado por usar o chatbot. Mantenha-se seguro e siga as orientações oficiais!")
            break

        respostas_para_imprimir = []
        acao_principal_realizada_nesta_rodada = False

        # --- Lógica de Continuação da Conversa (Estado) ---
        if estado_conversa.get("localizacao_perguntada") and not estado_conversa.get("local_informado"):
            estado_conversa["local_informado"] = usuario_input # Assume que a entrada é a localização
            respostas_para_imprimir.append(f"🤖 Chatbot: Entendido, sua localização de referência é '{usuario_input}'.")
            if estado_conversa.get("desastre_atual"):
                resposta_abrigos = agente_abrigos_sugerir_llm(estado_conversa["desastre_atual"], estado_conversa["local_informado"])
                respostas_para_imprimir.append(resposta_abrigos)
                if "⚠️" not in resposta_abrigos: # Se não deu erro e o agente de abrigos respondeu
                    respostas_para_imprimir.extend(agente_verificacao_seguranca_perguntar_txt())
            acao_principal_realizada_nesta_rodada = True

        elif estado_conversa.get("verificacao_seguranca_perguntada") and estado_conversa.get("usuario_seguro_confirmado") is None:
            resposta_status_seguranca = agente_processar_resposta_seguranca_llm(usuario_input)
            respostas_para_imprimir.append(resposta_status_seguranca)
            if estado_conversa.get("usuario_seguro_confirmado"):
                respostas_para_imprimir.append("\n🤖 Se precisar de mais alguma coisa, como informações de preparo ou suporte pós-desastre, me diga!")
            elif estado_conversa.get("usuario_seguro_confirmado") is False: # Explicitamente não seguro
                 respostas_para_imprimir.append("Por favor, tente contatar a Defesa Civil (199) ou Bombeiros (193) se estiver em perigo imediato.")
            # Resetar para permitir nova pergunta principal ou continuação sobre planejamento
            estado_conversa["localizacao_perguntada"] = False
            estado_conversa["verificacao_seguranca_perguntada"] = False
            acao_principal_realizada_nesta_rodada = True


        # --- Lógica de Nova Interação ou Intenção Principal ---
        if not acao_principal_realizada_nesta_rodada:
            # Se não caiu em uma continuação de estado, processa como nova intenção
            # resetar_estado_conversa() # Resetar completamente pode ser muito abrupto se o usuário só mudou de leve o tópico.
                                     # Vamos resetar apenas os flags de perguntas sequenciais se uma nova intenção principal for detectada.

            desastre_identificado, tipo_ajuda = agente_roteador_intencao(usuario_input)
            estado_conversa["desastre_atual"] = desastre_identificado if desastre_identificado else estado_conversa.get("desastre_atual")


            if tipo_ajuda == "desconhecida" and not desastre_identificado:
                respostas_para_imprimir.append("🤖 Chatbot: Desculpe, não entendi completamente. Poderia especificar sobre qual tipo de desastre (enchente, deslizamento, incêndio) ou que tipo de ajuda você precisa?")
            elif not estado_conversa.get("desastre_atual") and tipo_ajuda != "desconhecida":
                 respostas_para_imprimir.append(f"🤖 Chatbot: Sobre qual tipo de desastre (enchente, deslizamento, incêndio) você gostaria de falar em relação a '{tipo_ajuda}'?")
            else:
                # Temos um desastre (ou genérico) e um tipo de ajuda
                # Resetar flags de perguntas sequenciais para o caso de uma nova consulta sobre o mesmo desastre
                estado_conversa["localizacao_perguntada"] = False
                estado_conversa["local_informado"] = None
                estado_conversa["sugestoes_abrigo_dadas"] = False
                estado_conversa["verificacao_seguranca_perguntada"] = False
                # estado_conversa["usuario_seguro_confirmado"] = None # Manter o status de segurança se já confirmado? Ou resetar? Resetar é mais simples para nova consulta.
                estado_conversa["usuario_seguro_confirmado"] = None


                if tipo_ajuda == "imediata":
                    respostas_para_imprimir.append(agente_resposta_rapida_llm(estado_conversa["desastre_atual"]))
                    respostas_para_imprimir.extend(agente_localizacao_perguntar_txt())
                elif tipo_ajuda == "planejamento":
                    respostas_para_imprimir.append(agente_planejamento_suporte_llm(estado_conversa["desastre_atual"]))
                elif tipo_ajuda == "local_seguro": # Usuário perguntou diretamente sobre local seguro
                    respostas_para_imprimir.extend(agente_localizacao_perguntar_txt()) # Pergunta localização primeiro se ainda não tem
                    # O fluxo de resposta da localização cuidará de chamar o agente de abrigos
                elif tipo_ajuda == "geral" and estado_conversa.get("desastre_atual"):
                    respostas_para_imprimir.append(f"🤖 Chatbot: Certo, sobre {estado_conversa['desastre_atual']}. Aqui estão algumas ações imediatas:")
                    respostas_para_imprimir.append(agente_resposta_rapida_llm(estado_conversa["desastre_atual"]))
                    respostas_para_imprimir.extend(agente_localizacao_perguntar_txt()) # Inicia o fluxo completo
                elif tipo_ajuda == "desconhecida" and estado_conversa.get("desastre_atual"):
                    respostas_para_imprimir.append(f"🤖 Chatbot: Entendi que o assunto é sobre {estado_conversa['desastre_atual']}. Como posso te ajudar especificamente? (ações imediatas, preparo, locais seguros, etc.)")


        # Imprime as respostas
        if respostas_para_imprimir:
            for idx, resp_linha in enumerate(respostas_para_imprimir):
                if isinstance(resp_linha, list): # Se um agente retornou uma lista de linhas
                    for sub_linha in resp_linha:
                        print(sub_linha)
                else:
                    print(resp_linha)
                if idx < len(respostas_para_imprimir) - 1 and resp_linha and isinstance(resp_linha, str) and not resp_linha.startswith("---"):
                    print() # Adiciona um espaço entre blocos de texto, a menos que seja um título de agente

        elif not usuario_input.lower() == "sair": # Evita imprimir isso se o usuário apenas digitou 'sair'
            print("🤖 Chatbot: Não tenho uma resposta específica para isso no momento. Poderia tentar reformular ou perguntar sobre enchentes, deslizamentos ou incêndios?")

# --- Para iniciar o chatbot no Colab ---
if __name__ == "__main__":
    try:
        chatbot_desastres_naturais_llm()
    except Exception as e:
        print(f"Um erro inesperado ocorreu no chatbot: {e}")
        print("Por favor, verifique sua configuração de API Key e tente novamente.")

# Para executar diretamente no Colab após colar na célula,
# certifique-se de ter executado a instalação do pip e configurado a API Key.
# Então, você pode chamar:
# chatbot_desastres_naturais_llm()

✅ Chave de API configurada a partir dos secrets do Colab.


---
🤖 Bem-vindo ao Chatbot Inteligente de Assistência em Desastres Naturais!
   Sou um assistente virtual e usarei IA para tentar te ajudar.
   Lembre-se: Minhas informações são um auxílio. SEMPRE siga as orientações
   das autoridades oficiais (Defesa Civil, Bombeiros).
---
Como posso te ajudar hoje? (Para sair, digite 'sair')

🙎 Você: enchente
🤖 Acionando Agente de Resposta Rápida com modelo gemini-1.5-flash-latest...
🤖 Chatbot: Certo, sobre enchente. Aqui estão algumas ações imediatas:

➡️ **Evacue imediatamente:** Se ordenado pelas autoridades, deixe sua casa imediatamente. Siga as rotas de fuga indicadas.

➡️ **Procure abrigo em local seguro:** Se não conseguir evacuar, procure o andar mais alto do edifício, ou um local elevado e seguro, longe da área inundada.

➡️ **Desligue a eletricidade e o gás:** Evite choques elétricos e incêndios.

➡️ **Movimente seus pertences para locais altos:** Se possível, eleve móveis e objet